# Import Library

In [1]:
import numpy as np
import pandas as pd

import s2cell
from datetime import datetime as dt
from scipy.stats import chi2_contingency
from scipy.stats import chi2
from math import sin, cos
import itertools

In [2]:
pd.options.display.max_columns = 500

# Load Dataset

In [3]:
alerts = pd.read_csv('alers_clean2.csv')
data_train = pd.read_csv('../danthon2021/data_train.csv')
data_test = pd.read_csv('../danthon2021/data_test.csv')

In [4]:
alerts

,s2parent,s2token_15,date,hour,Type,SubType,N_Alerts,Road_Type,Reliability
0,2e68c263,2e68c2624,2020-09-29,17,{'JAM'},{'JAM_STAND_STILL_TRAFFIC'},32,{2},5.906250
1,2e68c263,2e68c2624,2020-10-08,16,{'JAM'},{'JAM_HEAVY_TRAFFIC'},18,{2},5.000000
2,2e68c263,2e68c2624,2020-10-20,16,{'JAM'},{'JAM_HEAVY_TRAFFIC'},31,{2},5.000000
3,2e68c263,2e68c2624,2020-11-09,17,{'JAM'},{'JAM_HEAVY_TRAFFIC'},31,{2},5.000000
4,2e68c263,2e68c2624,2020-11-13,17,{'JAM'},{'JAM_HEAVY_TRAFFIC'},42,{2},5.380952
...,...,...,...,...,...,...,...,...,...
47653,2e6a2741,2e6a27404,2020-11-15,18,{'JAM'},{'JAM'},29,{1},5.000000
47654,2e6a276d,2e6a276cc,2020-09-19,15,{'JAM'},{'JAM_HEAVY_TRAFFIC'},30,{2},5.000000
47655,2e6a2773,2e6a27734,2020-09-19,15,{'JAM'},{'JAM_HEAVY_TRAFFIC'},30,{2},5.000000
47656,2e6a2773,2e6a2773c,2020-09-19,15,{'JAM'},{'JAM_HEAVY_TRAFFIC'},31,{2},5.000000


In [5]:
type_expand = pd.DataFrame(alerts.Type.apply(eval).tolist(), columns=[f'Type_{x}' for x in range(3)])
subtype_expand = pd.DataFrame(alerts.SubType.apply(eval).tolist(), columns=[f'SubType_{x}' for x in range(6)])
road_type_expand = pd.DataFrame(alerts.Road_Type.apply(eval).tolist(), columns=[f'Road_Type_{x}' for x in range(3)])

In [6]:
alerts_clean = pd.concat([alerts, type_expand, subtype_expand, road_type_expand], axis=1)
alerts_clean

,s2parent,s2token_15,date,hour,Type,SubType,N_Alerts,Road_Type,Reliability,Type_0,Type_1,Type_2,SubType_0,SubType_1,SubType_2,SubType_3,SubType_4,SubType_5,Road_Type_0,Road_Type_1,Road_Type_2
0,2e68c263,2e68c2624,2020-09-29,17,{'JAM'},{'JAM_STAND_STILL_TRAFFIC'},32,{2},5.906250,JAM,None,None,JAM_STAND_STILL_TRAFFIC,None,None,None,None,None,2,NaN,NaN
1,2e68c263,2e68c2624,2020-10-08,16,{'JAM'},{'JAM_HEAVY_TRAFFIC'},18,{2},5.000000,JAM,None,None,JAM_HEAVY_TRAFFIC,None,None,None,None,None,2,NaN,NaN
2,2e68c263,2e68c2624,2020-10-20,16,{'JAM'},{'JAM_HEAVY_TRAFFIC'},31,{2},5.000000,JAM,None,None,JAM_HEAVY_TRAFFIC,None,None,None,None,None,2,NaN,NaN
3,2e68c263,2e68c2624,2020-11-09,17,{'JAM'},{'JAM_HEAVY_TRAFFIC'},31,{2},5.000000,JAM,None,None,JAM_HEAVY_TRAFFIC,None,None,None,None,None,2,NaN,NaN
4,2e68c263,2e68c2624,2020-11-13,17,{'JAM'},{'JAM_HEAVY_TRAFFIC'},42,{2},5.380952,JAM,None,None,JAM_HEAVY_TRAFFIC,None,None,None,None,None,2,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47653,2e6a2741,2e6a27404,2020-11-15,18,{'JAM'},{'JAM'},29,{1},5.000000,JAM,None,None,JAM,None,None,None,None,None,1,NaN,NaN
47654,2e6a276d,2e6a276cc,2020-09-19,15,{'JAM'},{'JAM_HEAVY_TRAFFIC'},30,{2},5.000000,JAM,None,None,JAM_HEAVY_TRAFFIC,None,None,None,None,None,2,NaN,NaN
47655,2e6a2773,2e6a27734,2020-09-19,15,{'JAM'},{'JAM_HEAVY_TRAFFIC'},30,{2},5.000000,JAM,None,None,JAM_HEAVY_TRAFFIC,None,None,None,None,None,2,NaN,NaN
47656,2e6a2773,2e6a2773c,2020-09-19,15,{'JAM'},{'JAM_HEAVY_TRAFFIC'},31,{2},5.000000,JAM,None,None,JAM_HEAVY_TRAFFIC,None,None,None,None,None,2,NaN,NaN


In [7]:
alerts_clean2 = alerts_clean.drop(['Type', 'SubType', 'Road_Type'], axis=1)

In [73]:
alerts_clean.to_csv('alerts_clean3.csv', index=True)

In [8]:
train_clean = data_train.copy()
train_clean[['s2token_15', 'date', 'hour']] = train_clean.Ids.str.split('_', expand=True)
train_clean['hour'] = train_clean['hour'].astype(int)
train_clean['s2parent'] = train_clean['s2token_15'].apply(lambda x: s2cell.token_to_parent_token(x))

train_clean

,Ids,Labels,s2token_15,date,hour,s2parent
0,2e69e9384_2020-10-06_13,True,2e69e9384,2020-10-06,13,2e69e939
1,2e6992c7c_2020-10-02_17,True,2e6992c7c,2020-10-02,17,2e6992c7
2,2e69ef474_2020-09-13_19,True,2e69ef474,2020-09-13,19,2e69ef47
3,2e69c5fd4_2020-10-10_15,True,2e69c5fd4,2020-10-10,15,2e69c5fd
4,2e6992134_2020-09-12_11,True,2e6992134,2020-09-12,11,2e699213
...,...,...,...,...,...,...
71331,2e69eea5c_2020-11-09_10,False,2e69eea5c,2020-11-09,10,2e69eea5
71332,2e69c5944_2020-10-27_12,True,2e69c5944,2020-10-27,12,2e69c595
71333,2e69f2cd4_2020-11-07_14,True,2e69f2cd4,2020-11-07,14,2e69f2cd
71334,2e68e64e4_2020-09-23_9,False,2e68e64e4,2020-09-23,9,2e68e64f


In [9]:
merge_train = train_clean.merge(alerts_clean2, how='left', on=['s2parent', 's2token_15', 'date', 'hour'])
merge_train

,Ids,Labels,s2token_15,date,hour,s2parent,N_Alerts,Reliability,Type_0,Type_1,Type_2,SubType_0,SubType_1,SubType_2,SubType_3,SubType_4,SubType_5,Road_Type_0,Road_Type_1,Road_Type_2
0,2e69e9384_2020-10-06_13,True,2e69e9384,2020-10-06,13,2e69e939,26.0,5.0,JAM,None,None,JAM,JAM_STAND_STILL_TRAFFIC,None,None,None,None,7.0,NaN,NaN
1,2e6992c7c_2020-10-02_17,True,2e6992c7c,2020-10-02,17,2e6992c7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2e69ef474_2020-09-13_19,True,2e69ef474,2020-09-13,19,2e69ef47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2e69c5fd4_2020-10-10_15,True,2e69c5fd4,2020-10-10,15,2e69c5fd,32.0,5.5,JAM,None,None,JAM_STAND_STILL_TRAFFIC,None,None,None,None,None,6.0,NaN,NaN
4,2e6992134_2020-09-12_11,True,2e6992134,2020-09-12,11,2e699213,31.0,5.0,JAM,None,None,JAM_STAND_STILL_TRAFFIC,None,None,None,None,None,7.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71331,2e69eea5c_2020-11-09_10,False,2e69eea5c,2020-11-09,10,2e69eea5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71332,2e69c5944_2020-10-27_12,True,2e69c5944,2020-10-27,12,2e69c595,31.0,5.0,JAM,None,None,JAM_HEAVY_TRAFFIC,None,None,None,None,None,2.0,NaN,NaN
71333,2e69f2cd4_2020-11-07_14,True,2e69f2cd4,2020-11-07,14,2e69f2cd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71334,2e68e64e4_2020-09-23_9,False,2e68e64e4,2020-09-23,9,2e68e64f,570.0,6.0,ROAD_CLOSED,None,None,ROAD_CLOSED_EVENT,None,None,None,None,None,7.0,NaN,NaN


In [10]:
test_clean = data_test.copy()
test_clean[['s2token_15', 'date', 'hour']] = test_clean.Ids.str.split('_', expand=True)
test_clean['hour'] = test_clean['hour'].astype(int)
test_clean['s2parent'] = test_clean['s2token_15'].apply(lambda x: s2cell.token_to_parent_token(x))

test_clean

,Ids,s2token_15,date,hour,s2parent
0,2e6992a84_2020-11-25_18,2e6992a84,2020-11-25,18,2e6992a9
1,2e68e62f4_2020-11-29_20,2e68e62f4,2020-11-29,20,2e68e62f
2,2e68e81a4_2020-11-27_10,2e68e81a4,2020-11-27,10,2e68e81b
3,2e69eec04_2020-11-24_7,2e69eec04,2020-11-24,7,2e69eec1
4,2e698e4a4_2020-11-27_8,2e698e4a4,2020-11-27,8,2e698e4b
...,...,...,...,...,...
13836,2e68dd414_2020-11-26_5,2e68dd414,2020-11-26,5,2e68dd41
13837,2e698541c_2020-11-24_22,2e698541c,2020-11-24,22,2e698541
13838,2e69e8e0c_2020-11-24_10,2e69e8e0c,2020-11-24,10,2e69e8e1
13839,2e699a1cc_2020-11-24_18,2e699a1cc,2020-11-24,18,2e699a1d


In [11]:
merge_test = test_clean.merge(alerts_clean2, how='left', on=['s2parent', 's2token_15', 'date', 'hour'])
merge_test

,Ids,s2token_15,date,hour,s2parent,N_Alerts,Reliability,Type_0,Type_1,Type_2,SubType_0,SubType_1,SubType_2,SubType_3,SubType_4,SubType_5,Road_Type_0,Road_Type_1,Road_Type_2
0,2e6992a84_2020-11-25_18,2e6992a84,2020-11-25,18,2e6992a9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2e68e62f4_2020-11-29_20,2e68e62f4,2020-11-29,20,2e68e62f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2e68e81a4_2020-11-27_10,2e68e81a4,2020-11-27,10,2e68e81b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2e69eec04_2020-11-24_7,2e69eec04,2020-11-24,7,2e69eec1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2e698e4a4_2020-11-27_8,2e698e4a4,2020-11-27,8,2e698e4b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13836,2e68dd414_2020-11-26_5,2e68dd414,2020-11-26,5,2e68dd41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13837,2e698541c_2020-11-24_22,2e698541c,2020-11-24,22,2e698541,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13838,2e69e8e0c_2020-11-24_10,2e69e8e0c,2020-11-24,10,2e69e8e1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13839,2e699a1cc_2020-11-24_18,2e699a1cc,2020-11-24,18,2e699a1d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
alerts_token = pd.DataFrame(alerts.s2token_15.unique(), columns=['s2token'])
alerts_token

,s2token
0,2e68c2624
1,2e68c262c
2,2e68c2814
3,2e68c2824
4,2e68c284c
...,...
4336,2e6a27404
4337,2e6a276cc
4338,2e6a27734
4339,2e6a2773c


In [17]:
train_token = pd.DataFrame(data_train.Ids.str.split('_').apply(lambda x: x[0]).unique(), columns=['s2token'])
train_token['valid'] = 1
train_token

,s2token,valid
0,2e69e9384,1
1,2e6992c7c,1
2,2e69ef474,1
3,2e69c5fd4,1
4,2e6992134,1
...,...,...
4555,2e698970c,1
4556,2e68c2b14,1
4557,2e699ad3c,1
4558,2e698d5b4,1


In [18]:
test_token = pd.DataFrame(data_test.Ids.str.split('_').apply(lambda x: x[0]).unique(), columns=['s2token'])
test_token['valid'] = 1
test_token

,s2token,valid
0,2e6992a84,1
1,2e68e62f4,1
2,2e68e81a4,1
3,2e69eec04,1
4,2e698e4a4,1
...,...,...
1953,2e69e9ecc,1
1954,2e699a19c,1
1955,2e698c924,1
1956,2e68e8a04,1
